# Photonic Matrices

## Imports

In [1]:
# standard library
from collections import OrderedDict

# photontorch
import torch
import photontorch as pt

# other
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange

## Settings

In [2]:
%matplotlib inline
DEVICE = 'cpu'
np.random.seed(0)
torch.manual_seed(0)
np.set_printoptions(precision=2, suppress=True)
env = pt.Environment(frequency_domain=True, num_timesteps=1, enable_grad=True)
pt.set_environment(env);

## Unitary Matrices

A unitary matrix is a matrix $U$ for which
\begin{align*}
U\cdot U^\dagger = U^\dagger \cdot U = I
\end{align*}

A unitary matrix is easily implemented in photonics. Indeed, according to the paper *"[Experimental Realization of Any Discrete Unitary Matrix](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.73.58)"* by Reck et. al., Any unitary matrix can be written as a combination of phase shifters and directional couplers with variable coupling (or MZI's) (Figure (a))

However, there exists an alternative approach to achieve any unitary operation, first proposed by Clements et. al. : [Optimal design for universal multiport interferometers](https://www.osapublishing.org/optica/abstract.cfm?uri=optica-3-12-1460) (Figure (b))

![Unitary Matrix Paper](images/clements.jpeg)


## 2x2 Unitary matrix (Reck)

### Functions

In [3]:
def array(tensor):
    arr = tensor.data.cpu().numpy()
    if arr.shape[0] == 2:
        arr = arr[0] + 1j * arr[1]
    return arr

def tensor(array):
    if array.dtype == np.complex64 or array.dtype == np.complex128:
        array = np.stack([np.real(array), np.imag(array)])
    return torch.tensor(array, dtype=torch.get_default_dtype(), device=DEVICE)

def rand_phase():
    return float(2*np.pi*np.random.rand())

class Network(pt.Network):
    def handle_source(self, matrix, **kwargs):
        expanded_matrix = matrix[:,None,None,:,:]
        a,b,c,d,e = expanded_matrix.shape
        expanded_matrix = torch.cat([
            expanded_matrix,
            torch.zeros((a,b,c,self.nmc-d,e), device=expanded_matrix.device),
        ], -2)
        return expanded_matrix
    def forward(self, matrix):
        ''' matrix shape = (2, num_sources, num_sources)'''
        result = super(Network, self).forward(matrix, power=False)
        return result[:,0,0,:,:]
    def count_params(self):
        num_params = 0
        for p in self.parameters():
            num_params += int(np.prod(p.shape))
        return num_params

def unitary_matrix(m,n):
    real_part = np.random.rand(m,n)
    imag_part = np.random.rand(m,n)
    complex_matrix = real_part + 1j*imag_part
    if m >= n:
        unitary_matrix, _, _ = np.linalg.svd(complex_matrix, full_matrices = False)
    else:
        _, _, unitary_matrix = np.linalg.svd(complex_matrix, full_matrices = False)
    return unitary_matrix

### Define Network

In [4]:
class Network2x2(Network):
    def __init__(self):
        super(Network2x2, self).__init__()
        self.s1 = pt.Source()
        self.s2 = pt.Source()
        self.d1 = pt.Detector()
        self.d2 = pt.Detector()
        self.mzi = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.wg1 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.wg2 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.link('s1:0', '0:mzi:1', '0:wg1:1', '0:d1')
        self.link('s2:0', '3:mzi:2', '0:wg2:1', '0:d2')
        
nw2x2 = Network2x2().to(DEVICE).initialize()

### Check unitarity

To see which unitary matrix the network represents, we search for the result of the propagation of an identity matrix through the network. The power flag was set to false, as we are interested in the full complex output of the system. To show that this matrix is indeed unitary, we multiply with its conjugate transpose:

In [5]:
def check_unitarity(nw):
    matrix = tensor(np.eye(nw.num_sources) + 0j)
    result = array(nw(matrix))
    print(result@result.T.conj())

check_unitarity(nw2x2)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.-0.j]]


### Check Universality

However, it would be more interesting if we can show that this network can act like *any* unitary matrix. We will now train the network to be equal to another unitary matrix by using the unitary property $U\cdot U^\dagger=I$: we will train the network to obtain $I$ with $U_0^\dagger$ as input.

In [6]:
def check_universality(nw, num_epochs=500, lr=0.1):
    matrix_to_approximate = unitary_matrix(nw.num_sources, nw.num_sources)
    matrix_input = tensor(matrix_to_approximate.T.conj().copy())
    eye = tensor(np.eye(nw.num_sources) + 0j)
    optimizer = torch.optim.Adam(nw.parameters(), lr=lr)
    lossfunc = torch.nn.MSELoss()
    epochs = trange(num_epochs)
    for i in epochs:
        optimizer.zero_grad()
        result = nw(matrix_input)
        loss = lossfunc(result, eye)
        loss.backward()
        optimizer.step()
        epochs.set_postfix(loss=f'{loss.item():.7f}')
        if loss.item() < 1e-6:
            break

    matrix_approximated = array(nw(eye))
    print(matrix_approximated)
    print(matrix_to_approximate)

In [7]:
check_universality(nw2x2)


[[-0.29-0.62j  0.18+0.71j]
 [-0.34-0.65j -0.2 -0.65j]]
[[-0.29-0.62j  0.18+0.71j]
 [-0.34-0.65j -0.2 -0.65j]]


## 3x3 Unitary Matrix (Reck)

In [8]:
class Reck3x3(Network):
    def __init__(self):
        super(Reck3x3, self).__init__()
        self.s1 = pt.Source()
        self.s2 = pt.Source()
        self.s3 = pt.Source()
        self.d1 = pt.Detector()
        self.d2 = pt.Detector()
        self.d3 = pt.Detector()
        self.mzi1 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.mzi2 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.mzi3 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.wg1 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.wg2 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.wg3 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.link("s1:0",                         "0:mzi1:1",                        "0:d1")
        self.link("s2:0",             "0:mzi2:1", "3:mzi1:2", "0:mzi3:1",            "0:d2")
        self.link("s3:0", "0:wg1:1",  "3:mzi2:2", "0:wg2:1",  "3:mzi3:2", "0:wg3:1", "0:d3")
reck3x3 = Reck3x3().to(DEVICE).initialize()

### Check Unitarity

In [9]:
check_unitarity(reck3x3)

[[ 1.-0.j -0.+0.j  0.-0.j]
 [-0.-0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j]]


### Check Universality

## 3x3 Unitary Matrix (Clements)

In [10]:
class Clements3x3(Network):
    def __init__(self):
        super(Clements3x3, self).__init__()
        self.s1 = pt.Source()
        self.s2 = pt.Source()
        self.s3 = pt.Source()
        self.d1 = pt.Detector()
        self.d2 = pt.Detector()
        self.d3 = pt.Detector()
        self.mzi1 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.mzi2 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.mzi3 = pt.Mzi(length=0, phi=rand_phase(), theta=rand_phase(), trainable=True)
        self.wg1 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.wg2 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.wg3 = pt.Waveguide(length=0, phase=rand_phase(), trainable=True)
        self.link("s1:0", "0:mzi1:1",             "0:mzi3:1", "0:wg1:1", "0:d1")
        self.link("s2:0", "3:mzi1:2", "0:mzi2:1", "3:mzi3:2", "0:wg2:1", "0:d2")
        self.link("s3:0",             "3:mzi2:2",             "0:wg3:1", "0:d3")
clem3x3 = Clements3x3().to(DEVICE).initialize()

### Check Unitarity

In [11]:
check_unitarity(clem3x3)

[[1.-0.j 0.-0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j]
 [0.-0.j 0.-0.j 1.+0.j]]


### Check Universality

## NxN Unitary Matrix (Reck)

Creating those networks is quite cumbersome. However they are also implemented by photontorch, which then handles the creation of the networks:

In [12]:
reck2x2 = pt.ReckNxN(N=2).to(DEVICE).terminate().initialize()
reck5x5 = pt.ReckNxN(N=5).to(DEVICE).terminate().initialize()
# quick monkeypatch to have the same behavior as the classes defined above
reck5x5.__class__ = Network

### Check Unitarity

In [13]:
check_unitarity(reck5x5)

[[ 1.+0.j  0.+0.j  0.-0.j -0.-0.j -0.-0.j]
 [ 0.-0.j  1.-0.j  0.-0.j  0.-0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-0.+0.j  0.+0.j  0.+0.j  1.-0.j  0.+0.j]
 [-0.+0.j  0.-0.j  0.+0.j  0.-0.j  1.-0.j]]


### Check Universality

In [14]:
check_universality(reck5x5)


[[-0.13-0.32j -0.17+0.09j -0.4 -0.24j  0.56+0.54j  0.1 +0.11j]
 [-0.32-0.42j  0.1 +0.52j  0.29-0.11j -0.22-0.02j  0.47-0.27j]
 [-0.39-0.38j -0.49-0.11j -0.24-0.02j -0.28-0.22j -0.48-0.18j]
 [-0.11-0.4j   0.55-0.27j -0.37+0.29j -0.3 +0.01j  0.11+0.36j]
 [-0.2 -0.31j  0.22-0.08j  0.63-0.04j  0.36-0.04j -0.42+0.31j]]
[[-0.13-0.32j -0.17+0.09j -0.4 -0.24j  0.56+0.54j  0.11+0.11j]
 [-0.32-0.42j  0.1 +0.52j  0.29-0.11j -0.22-0.02j  0.47-0.27j]
 [-0.39-0.38j -0.49-0.11j -0.24-0.02j -0.28-0.22j -0.48-0.18j]
 [-0.12-0.4j   0.55-0.26j -0.37+0.29j -0.3 +0.01j  0.11+0.36j]
 [-0.2 -0.32j  0.22-0.08j  0.63-0.04j  0.36-0.04j -0.42+0.31j]]


## NxN Unitary Matrix (Clements)

In [15]:
clem5x5 = pt.ClementsNxN(N=5).to(DEVICE).terminate().initialize()
clem6x6 = pt.ClementsNxN(N=6).to(DEVICE).terminate().initialize()
# quick monkeypatch to have the same behavior as the classes defined above
clem5x5.__class__ = clem6x6.__class__ = Network

### Check Unitarity

In [16]:
check_unitarity(clem5x5)
check_unitarity(clem6x6)

[[ 1.+0.j  0.+0.j -0.+0.j -0.-0.j -0.+0.j]
 [ 0.-0.j  1.+0.j -0.-0.j  0.-0.j  0.+0.j]
 [-0.-0.j -0.+0.j  1.-0.j  0.-0.j  0.+0.j]
 [-0.+0.j  0.+0.j  0.+0.j  1.+0.j -0.-0.j]
 [-0.-0.j  0.-0.j  0.-0.j -0.+0.j  1.-0.j]]
[[ 1.-0.j -0.+0.j -0.+0.j -0.+0.j  0.-0.j -0.+0.j]
 [-0.-0.j  1.-0.j  0.-0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.-0.j  0.+0.j  1.-0.j  0.+0.j -0.-0.j  0.+0.j]
 [-0.-0.j -0.+0.j  0.-0.j  1.+0.j -0.-0.j  0.+0.j]
 [ 0.+0.j -0.-0.j -0.+0.j -0.+0.j  1.+0.j -0.+0.j]
 [-0.-0.j -0.-0.j  0.-0.j  0.+0.j -0.-0.j  1.+0.j]]


### Check Universality

In [17]:
check_universality(clem5x5, num_epochs=1000)
check_universality(clem6x6, num_epochs=1000)


[[-0.25-0.18j -0.04-0.77j -0.02+0.09j  0.18-0.47j -0.11+0.19j]
 [-0.33-0.48j -0.31+0.33j -0.15+0.24j  0.14+0.08j -0.59+0.01j]
 [-0.3 -0.27j  0.1 -0.32j  0.01-0.09j -0.39+0.7j   0.12+0.24j]
 [-0.28-0.38j  0.02+0.25j -0.3 -0.63j  0.07-0.27j  0.38+0.04j]
 [-0.14-0.41j  0.06+0.1j   0.38+0.52j  0.03-0.09j  0.54-0.3j ]]
[[-0.25-0.18j -0.04-0.77j -0.02+0.09j  0.18-0.47j -0.1 +0.19j]
 [-0.33-0.48j -0.31+0.33j -0.15+0.24j  0.14+0.08j -0.59+0.01j]
 [-0.3 -0.27j  0.1 -0.32j  0.01-0.09j -0.39+0.7j   0.12+0.25j]
 [-0.28-0.38j  0.02+0.25j -0.3 -0.63j  0.07-0.27j  0.38+0.04j]
 [-0.14-0.41j  0.06+0.1j   0.38+0.52j  0.03-0.09j  0.54-0.3j ]]



[[-0.34-0.27j  0.12-0.31j  0.  -0.04j -0.15+0.42j -0.05-0.68j -0.17-0.11j]
 [-0.22-0.32j  0.34-0.29j  0.48+0.13j -0.08+0.01j -0.13+0.51j -0.3 +0.15j]
 [-0.31-0.38j -0.12-0.11j -0.12+0.42j  0.27+0.06j  0.07+0.13j  0.67-0.08j]
 [-0.25-0.19j  0.01-0.19j -0.61-0.35j -0.18-0.25j -0.14+0.3j  -0.13-0.39j]
 [-0.37-0.23j -0.32+0.55j  0.13-0.1j  -0.49-0.12j -0.21-0.03j  0.09+0.27j]
 [-0.31-0.2j   0.25+0.41j -0.16-0.09j  0.58-0.17j  0.27-0.1j  -0.32+0.22j]]
[[-0.34-0.27j  0.12-0.31j  0.  -0.04j -0.15+0.42j -0.04-0.68j -0.17-0.11j]
 [-0.22-0.32j  0.34-0.29j  0.48+0.13j -0.08+0.01j -0.13+0.52j -0.3 +0.14j]
 [-0.31-0.38j -0.12-0.11j -0.12+0.42j  0.27+0.06j  0.07+0.13j  0.67-0.08j]
 [-0.25-0.18j  0.01-0.19j -0.61-0.35j -0.18-0.25j -0.14+0.29j -0.13-0.39j]
 [-0.37-0.23j -0.32+0.55j  0.13-0.1j  -0.49-0.12j -0.21-0.03j  0.09+0.27j]
 [-0.3 -0.2j   0.25+0.41j -0.16-0.09j  0.58-0.17j  0.27-0.1j  -0.32+0.22j]]
